In [24]:
# importing this library so the seed stays the same
import inspect

import Utils.not_random
import Utils.submission as submission
from Data.RecSys2022 import RecSys2022, RecSys2022URMType
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Evaluator import Evaluator

from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender


dataset = RecSys2022(feature_dummies=True)
dataset.build(type=RecSys2022URMType.ONE_INTERACTED)

dataset2 = RecSys2022(feature_dummies=True)
dataset2.build(type=RecSys2022URMType.SUM)

urm = dataset.get_urm()
icm = dataset.get_icm()
urm_summed = dataset2.get_urm()

urm_type = dataset.get_urm_type()

urm_train, urm_validation = split_train_in_two_percentage_global_sample(
    urm, train_percentage=0.8)
urm_train_summed, urm_validation_summed = split_train_in_two_percentage_global_sample(
    urm_summed, train_percentage=0.8)

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_validation_summed = EvaluatorHoldout(urm_validation_summed, cutoff_list=[10])

print('Dataset loaded successfully with URM type:', urm_type.name)

model = RP3betaRecommender(urm_train_summed)

fit_params = {'topK': 236,
 'beta': 0.3635362818322568,
 'alpha': 0.11616698155699168,
 'normalize_similarity': True,
 'implicit': True}

print(f'Fitting model {model.__class__.__name__}...')
print(f'Fit params: {fit_params}')
model.fit(**fit_params)

print('Computing evaluation...')

result_df, _ = evaluator_validation_summed.evaluateRecommender(model)

map_value = result_df.loc[10]["MAP"]
print(f'MAP@10: {map_value:.7f}')

Unzipping dataset...
Loading interactions...
Loading features...
Loading target ids...
Cleaning up...
Building URM and ICM with criteria ONE_INTERACTED...
Unzipping dataset...
Loading interactions...
Loading features...
Loading target ids...
Cleaning up...
Building URM and ICM with criteria SUM...
EvaluatorHoldout: Ignoring 738 ( 1.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 704 ( 1.7%) Users that have less than 1 test interactions
Dataset loaded successfully with URM type: ONE_INTERACTED
Fitting model RP3betaRecommender...
Fit params: {'topK': 236, 'beta': 0.3635362818322568, 'alpha': 0.11616698155699168, 'normalize_similarity': True, 'implicit': True}
RP3betaRecommender: Similarity column 19630 (100.0%), 2957.59 column/sec. Elapsed time 6.64 sec
Computing evaluation...


/Users/luistavares/Desktop/recommender-systems/Evaluation/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/luistavares/Desktop/recommender-systems/Evaluation/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 40925 (100.0%) in 17.07 sec. Users per second: 2397
MAP@10: 0.0248030


In [4]:
model = RP3betaRecommender(urm_train)

fit_params = {
    'alpha': 0.719514,
    'beta': 0.229898,
    'min_rating': 0,
    'topK': 80,
    'implicit': True,
    'normalize_similarity': True
}

print(f'Fitting model {model.__class__.__name__}...')
print(f'Fit params: {fit_params}')
model.fit(**fit_params)

result_df, _ = evaluator_validation.evaluateRecommender(model)

map_value = result_df.loc[10]["MAP"]
print(f'MAP@10: {map_value:.7f}')


Fitting model RP3betaRecommender...
Fit params: {'alpha': 0.719514, 'beta': 0.229898, 'min_rating': 0, 'topK': 80, 'implicit': True, 'normalize_similarity': True}
RP3betaRecommender: Similarity column 19630 (100.0%), 4744.53 column/sec. Elapsed time 4.14 sec
EvaluatorHoldout: Processed 40856 (100.0%) in 14.21 sec. Users per second: 2874
MAP@10: 0.0299137


In [16]:
# importing this library so the seed stays the same
import os
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample, split_train_in_two_percentage_user_wise
import Utils.not_random
from Data.RecSys2022 import RecSys2022, RecSys2022URMType
from Data_manager.split_functions.split_train_validation_random_holdout import (
    split_train_in_two_percentage_global_sample,
    split_train_in_two_percentage_user_wise)
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

# building the dataset
# later we can call the build method to get different URM
dataset = RecSys2022()
dataset.build(type=RecSys2022URMType.SUM)

urm = dataset.get_urm()
icm = dataset.get_icm()

urm_train_validation, urm_test = split_train_in_two_percentage_global_sample(
    urm, train_percentage=0.80)
urm_train, urm_validation = split_train_in_two_percentage_global_sample(urm_train_validation, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

recommender_class = RP3betaRecommender

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                           evaluator_validation=evaluator_test)

hyperparameters_range_dictionary = {
    "topK": Integer(170, 350),
    "beta": Real(0.31, 0.4),
    "alpha": Real(0.09, 0.13),
    "normalize_similarity": Categorical([True]),
    "implicit": Categorical([True, False]),
}

recommender_input_args = SearchInputRecommenderArgs(
    # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_POSITIONAL_ARGS=[urm_train],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={
        'min_rating': 0
    },
    EARLYSTOPPING_KEYWORD_ARGS={},
)

recommender_input_args_last_test = SearchInputRecommenderArgs(
    # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_POSITIONAL_ARGS=[urm_test],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={
        'min_rating': 0
    },
    EARLYSTOPPING_KEYWORD_ARGS={},
)


output_folder_path = "result_experiments_4/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 100  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"
cutoff_to_optimize = 10

hyperparameterSearch.search(recommender_input_args,
                            recommender_input_args_last_test=recommender_input_args_last_test,
                            hyperparameter_search_space=hyperparameters_range_dictionary,
                            n_cases=n_cases,
                            n_random_starts=n_random_starts,
                            save_model="last",
                            output_folder_path=output_folder_path,  # Where to save the results
                            output_file_name_root=recommender_class.RECOMMENDER_NAME,  # How to call the files
                            metric_to_optimize=metric_to_optimize,
                            cutoff_to_optimize=cutoff_to_optimize,
                            )


Unzipping dataset...
Loading interactions...
Loading features...
Loading target ids...
Cleaning up...
Building URM and ICM with criteria SUM...
EvaluatorHoldout: Ignoring 1415 ( 3.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 700 ( 1.7%) Users that have less than 1 test interactions
Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 241, 'beta': 0.3546717916038427, 'alpha': 0.11692790815242463, 'normalize_similarity': True, 'implicit': False}
RP3betaRecommender: Similarity column 19630 (100.0%), 3105.74 column/sec. Elapsed time 6.32 sec


/Users/luistavares/Desktop/recommender-systems/Evaluation/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/luistavares/Desktop/recommender-systems/Evaluation/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 40929 (100.0%) in 16.89 sec. Users per second: 2423
SearchBayesianSkopt: New best config found. Config 0: {'topK': 241, 'beta': 0.3546717916038427, 'alpha': 0.11692790815242463, 'normalize_similarity': True, 'implicit': False} - results: PRECISION: 0.0448386, PRECISION_RECALL_MIN_DEN: 0.0720398, RECALL: 0.0665477, MAP: 0.0216716, MAP_MIN_DEN: 0.0343299, MRR: 0.1526458, NDCG: nan, F1: 0.0535776, HIT_RATE: 0.3111974, ARHR_ALL_HITS: 0.1806118, NOVELTY: 0.0052687, AVERAGE_POPULARITY: 0.2250630, DIVERSITY_MEAN_INTER_LIST: 0.8930490, DIVERSITY_HERFINDAHL: 0.9893027, COVERAGE_ITEM: 0.7668365, COVERAGE_ITEM_HIT: 0.0624045, ITEMS_IN_GT: 0.9984717, COVERAGE_USER: 0.9831848, COVERAGE_USER_HIT: 0.3059646, USERS_IN_GT: 0.9831848, DIVERSITY_GINI: 0.0641627, SHANNON_ENTROPY: 8.8336379, RATIO_DIVERSITY_HERFINDAHL: 0.9897260, RATIO_DIVERSITY_GINI: 0.1291562, RATIO_SHANNON_ENTROPY: 0.6724387, RATIO_AVERAGE_POPULARITY: 4.1774269, RATIO_NOVELTY: 0.4049291, 

Iteration No: 1 end

/Users/luistavares/Desktop/recommender-systems/Evaluation/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/luistavares/Desktop/recommender-systems/Evaluation/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 40929 (100.0%) in 16.71 sec. Users per second: 2450
SearchBayesianSkopt: New best config found. Config 1: {'topK': 228, 'beta': 0.36258119992876525, 'alpha': 0.10524815447364909, 'normalize_similarity': True, 'implicit': False} - results: PRECISION: 0.0448191, PRECISION_RECALL_MIN_DEN: 0.0719776, RECALL: 0.0664828, MAP: 0.0216856, MAP_MIN_DEN: 0.0343294, MRR: 0.1527303, NDCG: nan, F1: 0.0535426, HIT_RATE: 0.3110264, ARHR_ALL_HITS: 0.1807051, NOVELTY: 0.0052929, AVERAGE_POPULARITY: 0.2204348, DIVERSITY_MEAN_INTER_LIST: 0.8982732, DIVERSITY_HERFINDAHL: 0.9898251, COVERAGE_ITEM: 0.7717779, COVERAGE_ITEM_HIT: 0.0637290, ITEMS_IN_GT: 0.9984717, COVERAGE_USER: 0.9831848, COVERAGE_USER_HIT: 0.3057964, USERS_IN_GT: 0.9831848, DIVERSITY_GINI: 0.0663797, SHANNON_ENTROPY: 8.9151995, RATIO_DIVERSITY_HERFINDAHL: 0.9902487, RATIO_DIVERSITY_GINI: 0.1336189, RATIO_SHANNON_ENTROPY: 0.6786473, RATIO_AVERAGE_POPULARITY: 4.0915231, RATIO_NOVELTY: 0.4067887, 

Iteration No: 2 en

...

EvaluatorHoldout: Processed 40929 (100.0%) in 15.99 sec. Users per second: 2559
SearchBayesianSkopt: New best config found. Config 97: {'topK': 236, 'beta': 0.3635362818322568, 'alpha': 0.11616698155699168, 'normalize_similarity': True, 'implicit': True} - results: PRECISION: 0.0448777, PRECISION_RECALL_MIN_DEN: 0.0720478, RECALL: 0.0665552, MAP: 0.0217162, MAP_MIN_DEN: 0.0343677, MRR: 0.1527985, NDCG: nan, F1: 0.0536080, HIT_RATE: 0.3112707, ARHR_ALL_HITS: 0.1808834, NOVELTY: 0.0052852, AVERAGE_POPULARITY: 0.2224895, DIVERSITY_MEAN_INTER_LIST: 0.8959105, DIVERSITY_HERFINDAHL: 0.9895889, COVERAGE_ITEM: 0.7738156, COVERAGE_ITEM_HIT: 0.0632705, ITEMS_IN_GT: 0.9984717, COVERAGE_USER: 0.9831848, COVERAGE_USER_HIT: 0.3060367, USERS_IN_GT: 0.9831848, DIVERSITY_GINI: 0.0662408, SHANNON_ENTROPY: 8.8877752, RATIO_DIVERSITY_HERFINDAHL: 0.9900123, RATIO_DIVERSITY_GINI: 0.1333393, RATIO_SHANNON_ENTROPY: 0.6765597, RATIO_AVERAGE_POPULARITY: 4.1296611, RATIO_NOVELTY: 0.4061989, 

Iteration No: 98 en

/Users/luistavares/Desktop/recommender-systems/Evaluation/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/luistavares/Desktop/recommender-systems/Evaluation/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 40929 (100.0%) in 15.91 sec. Users per second: 2573
SearchBayesianSkopt: Config 98 is suboptimal. Config: {'topK': 236, 'beta': 0.3371932491614555, 'alpha': 0.12261502878600405, 'normalize_similarity': True, 'implicit': False} - results: PRECISION: 0.0447898, PRECISION_RECALL_MIN_DEN: 0.0721198, RECALL: 0.0666415, MAP: 0.0216821, MAP_MIN_DEN: 0.0343806, MRR: 0.1529692, NDCG: nan, F1: 0.0535731, HIT_RATE: 0.3114662, ARHR_ALL_HITS: 0.1808496, NOVELTY: 0.0052456, AVERAGE_POPULARITY: 0.2281103, DIVERSITY_MEAN_INTER_LIST: 0.8902143, DIVERSITY_HERFINDAHL: 0.9890193, COVERAGE_ITEM: 0.7501274, COVERAGE_ITEM_HIT: 0.0614875, ITEMS_IN_GT: 0.9984717, COVERAGE_USER: 0.9831848, COVERAGE_USER_HIT: 0.3062288, USERS_IN_GT: 0.9831848, DIVERSITY_GINI: 0.0604790, SHANNON_ENTROPY: 8.7605796, RATIO_DIVERSITY_HERFINDAHL: 0.9894425, RATIO_DIVERSITY_GINI: 0.1217411, RATIO_SHANNON_ENTROPY: 0.6668773, RATIO_AVERAGE_POPULARITY: 4.2339882, RATIO_NOVELTY: 0.4031584, 

Iteration No: 99 en

/Users/luistavares/Desktop/recommender-systems/Evaluation/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/luistavares/Desktop/recommender-systems/Evaluation/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 40929 (100.0%) in 16.03 sec. Users per second: 2553
SearchBayesianSkopt: Config 99 is suboptimal. Config: {'topK': 229, 'beta': 0.3848500000314843, 'alpha': 0.1284645235452362, 'normalize_similarity': True, 'implicit': True} - results: PRECISION: 0.0448631, PRECISION_RECALL_MIN_DEN: 0.0720089, RECALL: 0.0665133, MAP: 0.0216603, MAP_MIN_DEN: 0.0342392, MRR: 0.1521911, NDCG: nan, F1: 0.0535839, HIT_RATE: 0.3110020, ARHR_ALL_HITS: 0.1802713, NOVELTY: 0.0053181, AVERAGE_POPULARITY: 0.2176534, DIVERSITY_MEAN_INTER_LIST: 0.9006012, DIVERSITY_HERFINDAHL: 0.9900579, COVERAGE_ITEM: 0.7892511, COVERAGE_ITEM_HIT: 0.0642384, ITEMS_IN_GT: 0.9984717, COVERAGE_USER: 0.9831848, COVERAGE_USER_HIT: 0.3057724, USERS_IN_GT: 0.9831848, DIVERSITY_GINI: 0.0708456, SHANNON_ENTROPY: 8.9908917, RATIO_DIVERSITY_HERFINDAHL: 0.9904816, RATIO_DIVERSITY_GINI: 0.1426085, RATIO_SHANNON_ENTROPY: 0.6844092, RATIO_AVERAGE_POPULARITY: 4.0398975, RATIO_NOVELTY: 0.4087304, 

Iteration No: 100 end

In [17]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df

,topK,beta,alpha,normalize_similarity,implicit
0,241,0.354672,0.116928,True,False
1,228,0.362581,0.105248,True,False
2,201,0.330738,0.108665,True,False
3,296,0.354428,0.109498,True,True
4,224,0.369823,0.090462,True,True
...,...,...,...,...,...
95,268,0.392582,0.111366,True,False
96,259,0.323476,0.099535,True,False
97,236,0.363536,0.116167,True,True
98,236,0.337193,0.122615,True,False


In [18]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df

,,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
,cutoff,,,,,,,,,,,,,,,,,,,,,
0,10,0.044839,0.07204,0.066548,0.021672,0.03433,0.152646,NaN,0.053578,0.311197,0.180612,...,0.983185,0.305965,0.983185,0.064163,8.833638,0.989726,0.129156,0.672439,4.177427,0.404929
1,10,0.044819,0.071978,0.066483,0.021686,0.034329,0.15273,NaN,0.053543,0.311026,0.180705,...,0.983185,0.305796,0.983185,0.06638,8.915199,0.990249,0.133619,0.678647,4.091523,0.406789
2,10,0.044785,0.071933,0.066468,0.02162,0.034187,0.15254,NaN,0.053513,0.311344,0.180346,...,0.983185,0.306109,0.983185,0.060847,8.850459,0.99028,0.122482,0.673719,4.101855,0.405111
3,10,0.044931,0.07234,0.066855,0.021714,0.034443,0.153306,NaN,0.053744,0.31259,0.181216,...,0.983185,0.307334,0.983185,0.064198,8.740374,0.988812,0.129227,0.665339,4.326572,0.402825
4,10,0.044797,0.07195,0.066458,0.021692,0.034338,0.152829,NaN,0.053519,0.310733,0.180799,...,0.983185,0.305508,0.983185,0.068209,8.974556,0.990576,0.1373,0.683166,4.034461,0.408168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10,0.044853,0.07208,0.066595,0.021697,0.034317,0.152567,NaN,0.053603,0.3111,0.180685,...,0.983185,0.305869,0.983185,0.072634,8.972278,0.990117,0.146209,0.682992,4.098003,0.408376
96,10,0.044836,0.072221,0.066729,0.021684,0.034391,0.153131,NaN,0.053634,0.312321,0.180971,...,0.983185,0.30707,0.983185,0.057329,8.667816,0.98889,0.115401,0.659816,4.330667,0.400971
97,10,0.044878,0.072048,0.066555,0.021716,0.034368,0.152799,NaN,0.053608,0.311271,0.180883,...,0.983185,0.306037,0.983185,0.066241,8.887775,0.990012,0.133339,0.67656,4.129661,0.406199


In [19]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

{'topK': 236,
 'beta': 0.3635362818322568,
 'alpha': 0.11616698155699168,
 'normalize_similarity': True,
 'implicit': True}

In [20]:
import pandas as pd

In [21]:
pd.merge(search_metadata["hyperparameters_df"], pd.Series(pd.Series(search_metadata["result_on_validation_df"]["MAP"]).reset_index()["MAP"]), left_index=True, right_index=True).sort_values(by=["MAP"], ascending=False).head(30)

,topK,beta,alpha,normalize_similarity,implicit,MAP
97,236,0.363536,0.116167,True,True,0.021716
3,296,0.354428,0.109498,True,True,0.021714
61,302,0.373393,0.112264,True,False,0.021708
72,294,0.379515,0.092709,True,True,0.021705
8,228,0.365111,0.121685,True,False,0.021705
7,307,0.350671,0.108011,True,True,0.021704
34,302,0.368666,0.09,True,False,0.021704
93,232,0.380288,0.09,True,True,0.021703
5,256,0.36099,0.122925,True,True,0.021703
94,316,0.378944,0.109332,True,False,0.021703


In [22]:
params = {'topK': 236,
 'beta': 0.3635362818322568,
 'alpha': 0.11616698155699168,
 'normalize_similarity': True,
 'implicit': True}



In [ ]:
recommender = RP3betaRecommender(urm_train_summed)

evaluator_test_